# Pseudo Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_pseudodata.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.0.7


### Training

In [3]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

In [4]:
df = pd.read_csv('../datasets/pseudodata_train.csv')
x_train = df.drop('E', axis=1)
y_train = df['E']

In [5]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [6]:
model = iblm.fit(x_train, y_train, model_name = 'psedodata', file_path='./model_code/')

> Start of model creating.
Tokens Used: 7753
	Prompt Tokens: 7506
	Completion Tokens: 247
Successful Requests: 1
Total Cost (USD): $0.24


In [7]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():

        # Feature creation and data preprocessing
        A, B, C, D = row['A'], row['B'], row['C'], row['D']
        
        # Conditional branching and generation of new features
        E = A * B
        F = C * D
        G = A + C
        H = B + D
        
        # Sums and products of features
        I = E + F
        J = G * H
        
        # Linear relationships
        K = A * 0.5 + B * 0.3 + C * 0.2 + D * 0.1
        
        # As many formulas as possible
        L = (A * B * C * D) / (E * F * G * H)
        
        # Combine features to make a prediction
        y = K + L + I + J
        
        # Apply logistic function to convert y to probability
        y = 1 / (1 + np.exp(-y))
        output.append(y)
    return np.array(output)


### Prediction

In [8]:
df = pd.read_csv('../datasets/pseudodata_test.csv')
x_test = df.drop('E', axis=1)
y_test = df['E']

In [9]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [10]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.5443478260869565
Precision: 0.5490196078431373
Recall: 0.531951640759931
F1 score: 0.5403508771929825


ValueError: Input contains NaN.

### Prediction from external files


In [22]:
from .model_code import pseudodata

y_proba = pseudodata.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

ImportError: attempted relative import with no known parent package

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.415
Precision: 0.43103448275862066
Recall: 0.49504950495049505
F1 score: 0.4608294930875576
ROC-AUC: 0.35973597359735976


### Interpretation of results

In [57]:
description = iblm.interpret()

Tokens Used: 881
	Prompt Tokens: 537
	Completion Tokens: 344
Successful Requests: 1
Total Cost (USD): $0.036750000000000005


In [58]:
print(description)

- Data preprocessing:
    - Fill missing 'age' values with the median age.
    - Fill missing 'fare' values with the median fare.
    - Fill missing 'embarked' values with the mode (most frequent) of the 'embarked' column.

- Feature creation:
    - Create a new binary feature 'is_female' based on the 'sex' column.
    - Create a new binary feature 'is_child' based on the 'age' column.
    - Create a new binary feature 'is_adult_male' based on the 'adult_male' column.
    - Create a new binary feature 'is_alone' based on the 'alone' column.
    - Create new binary features 'is_first_class', 'is_second_class', and 'is_third_class' based on the 'pclass' column.
    - Create new binary features 'embarked_C', 'embarked_Q', and 'embarked_S' based on the 'embarked' column.

- Prediction logic:
    - Initialize a variable 'y' to 0.
    - Add or subtract weights to 'y' based on the created binary features.
    - Apply the logistic function (sigmoid) to 'y' to get the final prediction.
    - Ap